In [1]:
from gurobipy import *
import pandas as pd
import json
from tqdm import tqdm
import os

## Base Data Loading

Loading the data given for the project.

Note that the file `SallesDeCours.csv` has been slightly modified because the encoding was not correct.

In [2]:
def load_data(day):
    # day must be 18 to 21
    if day != 18 and day != 19 and day != 20 and day != 21:
        raise Exception('Invalid day')
    # Load data
    with open('data/students{}.json'.format(day)) as students_file:
        students_per_course = json.load(students_file)
    
    courses = pd.read_csv('data/Events{}.csv'.format(day), sep='\t', index_col=0)
    courses['Start'] = pd.to_datetime(courses['Start'])
    courses['End'] = pd.to_datetime(courses['End'])

    return students_per_course, courses


In [3]:

s_c18, c18 = load_data(18)
s_c19, c19 = load_data(19)
s_c20, c20 = load_data(20)
s_c21, c21 = load_data(21)

distances = pd.read_excel('data/distances.xlsx', index_col=0)
classrooms = pd.read_csv('data/SallesDeCours.csv', sep=';', index_col=0)

## Additional Data Computing

Six new data structures are computed from the base data:

- `distances`: update version containing the lower triangular matrix of distances between each pair of sectors
- `distance_between_classrooms`: a matrix containing the distance between each pair of classrooms
- `conflictsNN`: a matrix containing 1 if there's a conflicting schedule between each pair of courses
- `enrolledNN`: a matrix containing the number of students enrolled in each course
- `courses_per_studentNN`: a dictionary containing the courses followed by each student sorted by the start time
- `flowsNN`: a matrix containing the flows of students moving from a course to another

In [4]:
save = False

In [5]:
def compute_distance_between_classrooms(distances, classrooms):
    print('Computing distance between sectors ...')
    distances = distances.fillna(0)
    for i in tqdm(range(len(distances))):
        for j in range(0, i):
            distances.iloc[i, j] = distances.iloc[j, i]

    print('Computing distance between classrooms ...')
    distance_between_classrooms = pd.DataFrame(index=classrooms.index, columns=classrooms.index)
    for i in tqdm(range(len(classrooms))):
        building_i = classrooms.iloc[i]['Building']
        for j in range(len(classrooms)):
            building_j = classrooms.iloc[j]['Building']
            if building_i == building_j:
                distance_between_classrooms.iloc[i, j] = 0
            else:
                distance_between_classrooms.iloc[i, j] = distances.loc[building_i[0], building_j[0]]

    return distances, distance_between_classrooms

In [6]:
if save:
    distances, distance_between_classrooms = compute_distance_between_classrooms(distances, classrooms)

In [7]:
def time_intersect(e1, e2):
    # event 2 starts or ends during event 1
    if e1[0] < e2[0] < e1[1] or e1[0] < e2[1] < e1[1]:
        return 1
    # event 1 starts or ends during event 2
    elif e2[0] < e1[0] < e2[1] or e2[0] < e1[1] < e2[1]:
        return 1
    else:
        return 0
    
def compute_conflicts(courses):
    print('Computing conflicts between courses ...')
    # use index of courses to create double entry dataframe conflicts
    conflicts = pd.DataFrame(index=courses.index, columns=courses.index)
    for i in tqdm(range(len(courses))):
        for j in range(len(courses)):
            if i == j:
                conflicts.iloc[i, j] = 0
            else:
                conflicts.iloc[i, j] = time_intersect((courses.iloc[i]['Start'], courses.iloc[i]['End']), (courses.iloc[j]['Start'], courses.iloc[j]['End']))

    return conflicts

In [8]:
if save:
    conflicts18 = compute_conflicts(c18)
    conflicts19 = compute_conflicts(c19)
    conflicts20 = compute_conflicts(c20)
    conflicts21 = compute_conflicts(c21)

In [9]:
def compute_enrolled(students_per_course):
    print('Computing number of students enrolled in each course ...')
    enrolled = pd.DataFrame(index=students_per_course.keys(), columns=['Enrolled'])
    for course in tqdm(students_per_course.keys()):
        enrolled.loc[course, 'Enrolled'] = len(students_per_course[course])

    return enrolled

In [10]:
if save:
    enrolled18 = compute_enrolled(s_c18)
    enrolled19 = compute_enrolled(s_c19)
    enrolled20 = compute_enrolled(s_c20)
    enrolled21 = compute_enrolled(s_c21)

In [11]:
def compute_courses_per_student(students_per_course, courses):
    courses_per_student = {}
    print('Computing courses for each student ...')
    for course in tqdm(students_per_course):
        for student in students_per_course[course]:
            if student not in courses_per_student:
                courses_per_student[student] = [course]
            else:
                courses_per_student[student].append(course)

    print('Sorting by start time ...')
    for student in tqdm(courses_per_student):
        courses_per_student[student] = sorted(courses_per_student[student], key=lambda x: courses.loc[x]['Start'])

    return courses_per_student

In [12]:
if save:
    courses_per_student18 = compute_courses_per_student(s_c18, c18)
    courses_per_student19 = compute_courses_per_student(s_c19, c19)
    courses_per_student20 = compute_courses_per_student(s_c20, c20)
    courses_per_student21 = compute_courses_per_student(s_c21, c21)

In [13]:
def compute_flows(courses, courses_per_student):
    print('Computing student flows between courses ...')
    flows = pd.DataFrame(index=courses.index, columns=courses.index)
    flows = flows.fillna(0)
    
    for student in tqdm(courses_per_student):
        for i in range(len(courses_per_student[student])):
            for j in range(i+1, len(courses_per_student[student])):
                flows.loc[courses_per_student[student][i], courses_per_student[student][j]] += 1

    return flows

In [14]:
if save:
    flows18 = compute_flows(c18, courses_per_student18)
    flows19 = compute_flows(c19, courses_per_student19)
    flows20 = compute_flows(c20, courses_per_student20)
    flows21 = compute_flows(c21, courses_per_student21)

In [15]:
if save:
    if not os.path.exists('data/model_data'):
        os.makedirs('data/model_data')

    distances.to_csv('data/model_data/distances.csv')

    distance_between_classrooms.to_csv('data/model_data/distance_between_classrooms.csv')

    conflicts18.to_csv('data/model_data/conflicts18.csv')
    conflicts19.to_csv('data/model_data/conflicts19.csv')
    conflicts20.to_csv('data/model_data/conflicts20.csv')
    conflicts21.to_csv('data/model_data/conflicts21.csv')

    enrolled18.to_csv('data/model_data/enrolled18.csv')
    enrolled19.to_csv('data/model_data/enrolled19.csv')
    enrolled20.to_csv('data/model_data/enrolled20.csv')
    enrolled21.to_csv('data/model_data/enrolled21.csv')

    with open('data/model_data/courses_per_student18.json', 'w') as fp:
        json.dump(courses_per_student18, fp)
    with open('data/model_data/courses_per_student19.json', 'w') as fp:
        json.dump(courses_per_student19, fp)
    with open('data/model_data/courses_per_student20.json', 'w') as fp:
        json.dump(courses_per_student20, fp)
    with open('data/model_data/courses_per_student21.json', 'w') as fp:
        json.dump(courses_per_student21, fp)

    flows18.to_csv('data/model_data/flows18.csv')
    flows19.to_csv('data/model_data/flows19.csv')
    flows20.to_csv('data/model_data/flows20.csv')
    flows21.to_csv('data/model_data/flows21.csv')

    print('Done!')

## Data Loading

In [16]:
compute18 = False
compute19 = False
compute20 = False
compute21 = False

s_c18, c18 = load_data(18)
s_c19, c19 = load_data(19)
s_c20, c20 = load_data(20)
s_c21, c21 = load_data(21)

classrooms = pd.read_csv('data/SallesDeCours.csv', sep=';', index_col=0)

distances = pd.read_csv('data/model_data/distances.csv', index_col=0)

distance_between_classrooms = pd.read_csv('data/model_data/distance_between_classrooms.csv', index_col=0)

conflicts18 = pd.read_csv('data/model_data/conflicts18.csv', index_col=0)
conflicts19 = pd.read_csv('data/model_data/conflicts19.csv', index_col=0)
conflicts20 = pd.read_csv('data/model_data/conflicts20.csv', index_col=0)
conflicts21 = pd.read_csv('data/model_data/conflicts21.csv', index_col=0)

enrolled18 = pd.read_csv('data/model_data/enrolled18.csv', index_col=0)
enrolled19 = pd.read_csv('data/model_data/enrolled19.csv', index_col=0)
enrolled20 = pd.read_csv('data/model_data/enrolled20.csv', index_col=0)
enrolled21 = pd.read_csv('data/model_data/enrolled21.csv', index_col=0)

with open('data/model_data/courses_per_student18.json', 'r') as fp:
    courses_per_student18 = json.load(fp)
with open('data/model_data/courses_per_student19.json', 'r') as fp:
    courses_per_student19 = json.load(fp)
with open('data/model_data/courses_per_student20.json', 'r') as fp:
    courses_per_student20 = json.load(fp)
with open('data/model_data/courses_per_student21.json', 'r') as fp:
    courses_per_student21 = json.load(fp)

flows18 = pd.read_csv('data/model_data/flows18.csv', index_col=0)
flows19 = pd.read_csv('data/model_data/flows19.csv', index_col=0)
flows20 = pd.read_csv('data/model_data/flows20.csv', index_col=0)
flows21 = pd.read_csv('data/model_data/flows21.csv', index_col=0)

## Model Definition

In [17]:
if compute18:
    model18 = Model('room_optimizer18')
elif compute19:
    model19 = Model('room_optimizer19')
elif compute20:
    model20 = Model('room_optimizer20')
elif compute21:
    model21 = Model('room_optimizer21')

### Decision Variable

We'll use a variable $x_{ij}$ to indicate if the course $i$ is scheduled in the classroom $j$.

In [18]:
if compute18:
    x18 = model18.addVars(c18.index, classrooms.index, vtype=GRB.BINARY, name='x18')
elif compute19:
    x19 = model19.addVars(c19.index, classrooms.index, vtype=GRB.BINARY, name='x19')
elif compute20:
    x20 = model20.addVars(c20.index, classrooms.index, vtype=GRB.BINARY, name='x20')
elif compute21:
    x21 = model21.addVars(c21.index, classrooms.index, vtype=GRB.BINARY, name='x21')

### Objective Function


In [19]:
if compute18:
    model18.setObjective(0, GRB.MINIMIZE)
elif compute19:
    model19.setObjective(0, GRB.MINIMIZE)
elif compute20:
    model20.setObjective(0, GRB.MINIMIZE)
elif compute21:
    model21.setObjective(0, GRB.MINIMIZE)

### Constraints

In [20]:
if compute18:
    model18.addConstrs((x18.sum(ei, '*') == 1 for ei in c18.index), name='one_room18');
elif compute19:
    model19.addConstrs((x19.sum(ei, '*') == 1 for ei in c19.index), name='one_room19');
elif compute20:
    model20.addConstrs((x20.sum(ei, '*') == 1 for ei in c20.index), name='one_room20');
elif compute21:
    model21.addConstrs((x21.sum(ei, '*') == 1 for ei in c21.index), name='one_room21');

In [21]:
if compute18:
    for ei in tqdm(c18.index):
        for ej in c18.index:
            if conflicts18.loc[ei, ej] == 1:
                for k in classrooms.index:
                    model18.addConstr(x18[ei, k] + x18[ej, k] <= 1);
elif compute19:
    for ei in tqdm(c19.index):
        for ej in c19.index:
            if conflicts19.loc[ei, ej] == 1:
                for k in classrooms.index:
                    model19.addConstr(x19[ei, k] + x19[ej, k] <= 1);
elif compute20:
    for ei in tqdm(c20.index):
        for ej in c20.index:
            if conflicts20.loc[ei, ej] == 1:
                for k in classrooms.index:
                    model20.addConstr(x20[ei, k] + x20[ej, k] <= 1);
elif compute21:
    for ei in tqdm(c21.index):
        for ej in c21.index:
            if conflicts21.loc[ei, ej] == 1:
                for k in classrooms.index:
                    model21.addConstr(x21[ei, k] + x21[ej, k] <= 1);

In [22]:
if compute18:    
    for ei in tqdm(c18.index):
        for k in classrooms.index:
            model18.addConstr(x18[ei, k] * enrolled18.loc[ei, 'Enrolled'] <= 1.5*classrooms.loc[k, 'Capacity']);
elif compute19:
    for ei in tqdm(c19.index):
        for k in classrooms.index:
            model19.addConstr(x19[ei, k] * enrolled19.loc[ei, 'Enrolled'] <= 2*classrooms.loc[k, 'Capacity']);
elif compute20:
    for ei in tqdm(c20.index):
        for k in classrooms.index:
            model20.addConstr(x20[ei, k] * enrolled20.loc[ei, 'Enrolled'] <= 1.5*classrooms.loc[k, 'Capacity']);
elif compute21:
    for ei in tqdm(c21.index):
        for k in classrooms.index:
            model21.addConstr(x21[ei, k] * enrolled21.loc[ei, 'Enrolled'] <= 2*classrooms.loc[k, 'Capacity']);

### Optimize

In [23]:
if compute18:
    model18.optimize();
elif compute19:
    model19.optimize();
elif compute20:
    model20.optimize();
elif compute21:
    model21.optimize();

In [24]:
if compute18:
    allocation18 = pd.DataFrame(index=c18.index, columns=['Room'])
    for ei in c18.index:
        for k in classrooms.index:
            if x18[ei, k].x == 1:
                allocation18.loc[ei, 'Room'] = k
    allocation18.to_csv('data/model_data/allocation18.csv')

elif compute19:
    allocation19 = pd.DataFrame(index=c19.index, columns=['Room'])
    for ei in c19.index:
        for k in classrooms.index:
            if x19[ei, k].x == 1:
                allocation19.loc[ei, 'Room'] = k
    allocation19.to_csv('data/model_data/allocation19.csv')

elif compute20:
    allocation20 = pd.DataFrame(index=c20.index, columns=['Room'])
    for ei in c20.index:
        for k in classrooms.index:
            if x20[ei, k].x == 1:
                allocation20.loc[ei, 'Room'] = k
    allocation20.to_csv('data/model_data/allocation20.csv')

elif compute21:
    allocation21 = pd.DataFrame(index=c21.index, columns=['Room'])
    for ei in c21.index:
        for k in classrooms.index:
            if x21[ei, k].x == 1:
                allocation21.loc[ei, 'Room'] = k
    allocation21.to_csv('data/model_data/allocation21.csv')

### Results

Although this solution was obtained by fixing the objective function to 0 and optimizing the model, it is still a valid solution.

If we wanted to use the true objective function it would give the following result :

In [25]:
# 18-10-21
print('### 18-10-2021 ###')
allocation18 = pd.read_csv('data/model_data/allocation18.csv', index_col=0)

total_walking_distance18 = 0
for ei in tqdm(c18.index):
    for ej in c18.index:
        if ei != ej:
            total_walking_distance18 += flows18.loc[ei, ej] * distance_between_classrooms.loc[allocation18.loc[ei, 'Room'], allocation18.loc[ej, 'Room']]

total_walking_distance18_per_student = total_walking_distance18 / len(courses_per_student18)
print('Total walking distance for 18-10-21: ', total_walking_distance18)
print('Total walking distance per student for 18-10-21: ', total_walking_distance18_per_student)

# 19-10-21
print('\n### 19-10-2021 ###')
allocation19 = pd.read_csv('data/model_data/allocation19.csv', index_col=0)

total_walking_distance19 = 0
for ei in tqdm(c19.index):
    for ej in c19.index:
        if ei != ej:
            total_walking_distance19 += flows19.loc[ei, ej] * distance_between_classrooms.loc[allocation19.loc[ei, 'Room'], allocation19.loc[ej, 'Room']]

total_walking_distance19_per_student = total_walking_distance19 / len(courses_per_student19)
print('Total walking distance for 19-10-21: ', total_walking_distance19)
print('Total walking distance per student for 19-10-21: ', total_walking_distance19_per_student)

# 20-10-21
print('\n### 20-10-2021 ###')
allocation20 = pd.read_csv('data/model_data/allocation20.csv', index_col=0)

total_walking_distance20 = 0
for ei in tqdm(c20.index):
    for ej in c20.index:
        if ei != ej:
            total_walking_distance20 += flows20.loc[ei, ej] * distance_between_classrooms.loc[allocation20.loc[ei, 'Room'], allocation20.loc[ej, 'Room']]

total_walking_distance20_per_student = total_walking_distance20 / len(courses_per_student20)
print('Total walking distance for 20-10-21: ', total_walking_distance20)
print('Total walking distance per student for 20-10-21: ', total_walking_distance20_per_student)


# 21-10-21
print('\n### 21-10-2021 ###')
allocation21 = pd.read_csv('data/model_data/allocation21.csv', index_col=0)

total_walking_distance21 = 0
for ei in tqdm(c21.index):
    for ej in c21.index:
        if ei != ej:
            total_walking_distance21 += flows21.loc[ei, ej] * distance_between_classrooms.loc[allocation21.loc[ei, 'Room'], allocation21.loc[ej, 'Room']]

total_walking_distance21_per_student = total_walking_distance21 / len(courses_per_student21)
print('Total walking distance for 21-10-21: ', total_walking_distance21)
print('Total walking distance per student for 21-10-21: ', total_walking_distance21_per_student)

### 18-10-2021 ###


100%|██████████| 490/490 [00:10<00:00, 48.61it/s]


Total walking distance for 18-10-21:  1590695.0
Total walking distance per student for 18-10-21:  90.40094339622641

### 19-10-2021 ###


100%|██████████| 532/532 [00:11<00:00, 45.06it/s]


Total walking distance for 19-10-21:  4039884.0
Total walking distance per student for 19-10-21:  235.67168358417922

### 20-10-2021 ###


100%|██████████| 510/510 [00:10<00:00, 46.97it/s]


Total walking distance for 20-10-21:  2860262.0
Total walking distance per student for 20-10-21:  164.2412862474878

### 21-10-2021 ###


100%|██████████| 539/539 [00:12<00:00, 43.12it/s]

Total walking distance for 21-10-21:  2688294.0
Total walking distance per student for 21-10-21:  149.90765627613897
